In [ ]:
#@title Librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
from scipy.integrate import simpson, romberg, trapezoid

# Actividad 07: Integración

---
### Profesor: Juan Marcos Marín
### Nombre: Miguel Ángel Jaramillo Valencia
*Métodos computacionales 2024-II*

---

#1
* Implemente una función para el **método de integración de Romberg** definiendo un límite de tolerancia de 1e-8 y/o un máximo de iteraciones de 10.

* Encuentre la integral para

$$\int_0^{\pi/4} dx\, e^{3x}\cdot \sin(x)$$

* Imprima su resultado y compare los valores dados por `scipy.integrate.romberg`

* Finalmente, encuentre el valor del error, hallando el valor exacto usando `sympy`



In [ ]:
def Romberg(f, a, b, tol = 1e-8, max_iter = 10):

  h = 1
  R = np.zeros([max_iter, max_iter])

  R[0,0] = h/2 * (f(a) + f(b))

  integral = f(a) + f(b)

  for i in range(1,max_iter):

    h /= 2

    integral +=  2 * (f(h))

    R[i,0] =  h * (integral)

    for j in range(1, i+1):

      R[i,j] = R[i,j-1] + (R[i,j-1] - R[i-1, j-1]) / (4 ** j - 1)

    if abs(R[i, i] - R[i-1, i-1]) < tol:
      return R[i,i], R[:i+1, :i+1]

  return R[i, i], R[:i+1, :i+1]

In [ ]:
f = lambda x: np.exp(3 * x) * np.sin(x)
Romberg(f, 0, np.pi/4)

(0.016004336793355854,
 array([[3.73024427, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [5.87888047, 6.59509253, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [3.20131733, 2.30879628, 2.0230432 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.64600883, 1.12757266, 1.04882442, 1.03336063, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.83242195, 0.56122633, 0.52346991, 0.51513095, 0.51309867,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.41835572, 0.28033364, 0.26160746, 0.25745091, 0.2564404 ,
         0.25618951, 0.        , 0.        , 0.        , 0.        ],
        [0.20968955, 0.14013416, 0.13078753, 0.12871103, 0.12820616,
         0.12808081, 0.12804953, 0.        , 0.        , 0.        ],
    

#2

* Usando los *métodos trapezoidal compuesto*, *simpson 1/3* y de *medio punto* encuentre la siguiente integral,

$$\int_e^{1+e} dx\, \frac{1}{x\ln x}$$

* Luego, haga un estudio de la convergencia en términos del valor de $h$ o de los sub-intervalos de la función. ¿Cuál es mejor?


In [ ]:
def trap_comp(f, a, b, n):

  h = (b - a) / n
  integral = f(a) + f(b)

  for i in range(1, n):

    integral +=  2 * (f(a + i * h))

  return  (h/2) * integral

def simpson1_3(f, a, b, n):

  h = (b - a) / n
  integral = (f(a) + f(b))

  for i in range(1, n, 2):
    integral += 4 * f(a + i * h)

  for i in range(2, n, 2):
    integral += 2 * f(a + i * h)

  return integral * h / 3

def punto_medio(f, a, b, n):
  h = (b - a)/ n
  suma = 0
  for i in range(0, n):

   xi = a + i * h

   suma += f(xi + h / 2)

  return h * suma

In [ ]:
f = lambda x: 1 / (x * np.log(x))
x = sp.symbols('x')
f1 = 1 / (x * sp.log(x))
print(f'el valor de la integral usando trapezoides es: {trap_comp(f, np.e, np.e+1, 10)}')
print(f'el valor de la integral usando Simpson 1/3 es: {simpson1_3(f, np.e, np.e+1, 10 )}')
print(f'el valor de la integral usando punto medio es: {punto_medio(f, np.e, np.e+1, 10 )}')
print(f'el valor exacto de la integral es: {sp.integrate(f1,(x, np.e, np.e + 1))}')

el valor de la integral usando trapezoides es: 0.27265851808866515
el valor de la integral usando Simpson 1/3 es: 0.2725141807832367
el valor de la integral usando punto medio es: 0.27244159005549795
el valor exacto de la integral es: 0.272513880502583


Vamos a variar los $h$ para estudiar la convergencia de los métodos

In [ ]:
def trap_comp(f, a, b, n, h):

    integral = f(a) + f(b)

    for i in range(1, n):

      integral +=  2 * (f(a + i * h))

    return  (h/2) * integral

h = np.arange(1e-4, 0.1, 0.001)

T = trap_comp(f, np.e, np.e+1, 10, h)
C = abs(T - float(sp.integrate(f1,(x, np.e, np.e + 1))))
np.argmin(C), C.min(), h[np.argmin(C)]

(99, 0.001749014967892093, 0.09910000000000001)

In [ ]:
def simpson1_3(f, a, b, n, h):

  integral = (f(a) + f(b))

  for i in range(1, n, 2):
    integral += 4 * f(a + i * h)

  for i in range(2, n, 2):
    integral += 2 * f(a + i * h)

  return integral * h / 3

S = simpson1_3(f, np.e, np.e+1, 100, h)
C = abs(S - float(sp.integrate(f1,(x, np.e, np.e + 1))))
np.argmin(C), np.min(C),h[np.argmin(C)]

(10, 0.0020463043751273724, 0.0101)

In [ ]:
def punto_medio(f, a, b, n, h):

 suma = 0
 for i in range(0, n):

   xi = a + i * h

 suma += f(xi + h / 2)

 return h * suma

S = punto_medio(f, np.e, np.e+1, 100, h)
C = abs(S - float(sp.integrate(f1,(x, np.e, np.e + 1))))
np.argmin(C), C.min(),h[np.argmin(C)]

(59, 0.2693194858712926, 0.05910000000000001)

# Se observa que el error mínimo que pueden alcanzar los dos primeros métodos es de alrededor de 0.2 con los h correspondientemente presentados.

#3
Usando la siguiente función:



```python
def gauss_quad_standard(func, n):
    """
    Calcula la integral de una función en el intervalo [-1, 1]
    utilizando cuadratura gaussiana.

    Parameters:
    - func: La función a integrar.
    - n: Número de puntos para la cuadratura (grado del polinomio de Legendre).

    Returns:
    - Aproximación de la integral.
    """
    # Obtener raíces y pesos del polinomio de Legendre
    x, w = roots_legendre(n)

    # Evaluar la suma ponderada
    integral = np.sum(w * func(x))
    return integral
```

Modifique la función `gauss_quad_standard` de forma tal que no este restringida para $[-1,1]$ sino para cualquier intervalo $[a,b]$. Luego, encuentre la integral del *punto 2*.





In [ ]:
from scipy.special import roots_legendre
def gauss_quad_standard(func, a, b, n):

    xi, wi = roots_legendre(n)
    n1 = (b - a) / 2
    n2 = (a + b) / 2

    f = lambda x: func(n1 * x + n2)

    # Evaluar la suma ponderada
    integral = n1 * np.sum(wi * f(xi))
    return integral

In [ ]:
f = lambda x: 1 / (x * np.log(x))
gauss_quad_standard(f, np.e, np.e +1, 10)

0.27251388050258335

#4

Encuentra todas las raices para los polinomios de grado 3 y 4 de **Legendre** usando el Método de la Secante y Newton-Raphson.



```python
import sympy as sp
x = sp.Symbol('x')

# Polinomio de Legendre de grado n
Pn = sp.legendre(n, x)

```

y calcule los pesos $w_i$ de la cuadratura mediante la fórmula:
   $$
   w_i = \frac{2}{(1 - x_i^2) \left[P_n'(x_i)\right]^2},
   $$
   donde $P_n'(x)$ es la derivada del polinomio de Legendre $P_n(x)$.


In [ ]:
import sympy as sp
from scipy.optimize import newton, root_scalar
x = sp.Symbol('x')

# Polinomio de Legendre de grado n
P3 = sp.legendre(3, x)
P3_prima = sp.diff(P3, x)
P3 = sp.lambdify(x, P3)
P3_prima = sp.lambdify(x, P3_prima)

P4 = sp.legendre(4, x)
P4_prima = sp.diff(P4, x)
P4 = sp.lambdify(x, P4)
P4_prima = sp.lambdify(x, P4_prima)

raices = [newton(P3, i) for i in np.linspace(-1, 1, 10)]
raices_P3 = np.unique(np.round(raices, decimals = 6)) #Elimina raices que sean iguales en más de 6 decimales
print('las raíces del polinomio de legendre de grado 3 usando Newton son:',raices_P3)

raices1 = []
x0 = np.linspace(-1, 1, 10)
for i in x0:
  raiz = root_scalar(P3, x0=i, x1=i + 0.1, method='secant')
  raices1.append(raiz.root)

raices1_P3 = np.unique(np.round(raices1, decimals = 6))
print('las raíces del polinomio de Legendre de grado 3 usando Secante son:',raices1_P3)

las raíces del polinomio de legendre de grado 3 usando Newton son: [-0.774597 -0.        0.774597]
las raíces del polinomio de Legendre de grado 3 usando Secante son: [-0.774597 -0.        0.774597]


In [ ]:
w = []
w1 = []
for i in raices_P3:
  wi = 2 / ((1 - (i ** 2)) * (P3_prima(i)**2))
  w.append(wi)
print('los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 3 obtenido con Newton son:\n',w)

for i in raices1_P3:
  wi = 2 / ((1 - (i ** 2)) * (P3_prima(i)**2))
  w1.append(wi)
print('los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 3 obtenido con Secante son:\n',w1)


los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 3 obtenido con Newton son:
 [0.5555548438782123, 0.8888888888888888, 0.5555548438782123]
los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 3 obtenido con Secante son:
 [0.5555548438782123, 0.8888888888888888, 0.5555548438782123]


In [ ]:
raices2 = [newton(P4, i) for i in np.linspace(-1, 1, 10)]
raices_P4 = np.unique(np.round(raices2, decimals = 6))
print('las raíces del polinomio de legendre de grado 4 usando Newton son:',raices_P4)

raices3 = []
x0 = np.linspace(-1, 1, 10)
for i in x0:
  raiz = root_scalar(P4, x0=i, x1=i + 0.1, method='secant')
  raices3.append(raiz.root)

raices1_P4 = np.unique(np.round(raices3, decimals = 6))
print('las raíces del polinomio de Legendre de grado 4 usando Secante son:',raices1_P4)

las raíces del polinomio de legendre de grado 4 usando Newton son: [-0.861136 -0.339981  0.339981  0.861136]
las raíces del polinomio de Legendre de grado 4 usando Secante son: [-0.861136 -0.339981  0.339981  0.861136]


In [ ]:
w2 = []
w3 = []
for i in raices_P4:
  wj = 2 / ((1 - (i ** 2)) * (P4_prima(i)**2))
  w2.append(wj)
print('los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 4 obtenido con Newton son:\n',w2)

for i in raices1_P4:
  wj = 2 / ((1 - (i ** 2)) * (P4_prima(i)**2))
  w3.append(wj)
print('los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 4 obtenido con Secante son:\n',w3)


los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 4 obtenido con Newton son:
 [0.34785556744762186, 0.6521451767154952, 0.6521451767154952, 0.34785556744762186]
los pesos de la cuadratura correspondientes a las raices del polinomio de Legendre de grado 4 obtenido con Secante son:
 [0.34785556744762186, 0.6521451767154952, 0.6521451767154952, 0.34785556744762186]
